# Lasso selection for linear regression/classification

## Dataset

In [ ]:
def towdir(s):
    return (str('./datasets_book/'+s))

import deepglmlib.utils as utils
import numpy as np

In [ ]:
import importlib
importlib.reload(utils)

In [ ]:
import numpy as np
for filenamefix in {'','_train','_test'}:
    with open("".join([towdir('x_y'),filenamefix,'_450d_lasso.npz']), 'rb') as f:
        xy = np.load(f)
        namex_ = "".join(['x',filenamefix]); x_ = xy[namex_]
        namey_ = "".join(['y',filenamefix]); y_ = xy[namey_]
        print(namex_, x_.shape, namey_, y_.shape)

## Multiple regression without lasso

In [ ]:
import deepglmlib.utils as utils

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch

x_y_train_450d_lasso=np.load(towdir('./x_y_train_450d_lasso.npz'))
x_y_test_450d_lasso=np.load(towdir('./x_y_test_450d_lasso.npz'))

x_train = x_y_train_450d_lasso['x_train']
y_train = x_y_train_450d_lasso['y_train']

x_test = x_y_test_450d_lasso['x_test']
y_test = x_y_test_450d_lasso['y_test']


dataset_train = TensorDataset( torch.from_numpy(x_train.astype(np.float32)), 
                               torch.from_numpy(y_train.astype(np.float32)) )

dataset_test  = TensorDataset( torch.from_numpy(x_test.astype(np.float32)), 
                               torch.from_numpy(y_test.astype(np.float32)) )

In [ ]:
n_train, p_train = x_train.shape
n_test, p_test   = x_test.shape

n_train, p_train, n_test, p_test

In [ ]:
import torch
# import glmlib.utils as utils
from torch.utils.data import TensorDataset, DataLoader
dl_train            = DataLoader(dataset_train,shuffle=False,batch_size=10)#,num_workers=1,pin_memory=True)
dl_test             = DataLoader(dataset_test,shuffle=False,batch_size=10)#,num_workers=1,pin_memory=True)

In [ ]:
print("cuda.is_available()     = ", torch.cuda.is_available())
# print(torch.cuda.device(0)
# print(torch.cuda.device_count())
print("cuda.get_device_name(0) = ",torch.cuda.get_device_name(0))

### Training

The training function is now implementing with the gpu device for faster training, when available.

In [ ]:
device = torch.device("cuda:0" \
  if torch.cuda.is_available() else "cpu")

device

In [ ]:
import torch.nn as nn
import copy

#px = dataset_train.x.shape[1]
px = p_train

layers_regress = []
layers_regress.append(nn.Linear(px,1,bias=True))

model =  utils.GNLMRegression("LinearRegression",
                        copy.deepcopy(layers_regress))

model.to(device)
print(model)


nbmax_epoqs = 500
alpha_t     = 1e-4
debug_out   = 5
model.train()

loss      = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=alpha_t, momentum=0.0)
monitor   = utils.MyMonitorTest(model,loss,dl_train,dl_test,nbmax_epoqs,
                                debug_out,device=device)

# loss_s,tmax,monistopc  = utils.f_train_glmr(dl_train,model,optimizer,loss,monitor)

In [ ]:
loss_train_s,tmax,monistopc  = utils.f_train_glmr(dl_train,model,optimizer,
                                       loss,monitor,device=device,printed=2)

## Lasso selection for neural networks

In [ ]:
lambda_l1 = 0.020

In [ ]:
print(lambda_l1)

In [ ]:
def loss_yy_model(lossb,model):
    lossb_b_rg = lossb
    lossb_b_l1 = (torch.abs(list(model.parameters())[0])+0.000001).sum()
    loss_b = lossb_b_rg + lambda_l1 * lossb_b_l1
    return loss_b

In [ ]:
nbmax_epoqs = 350
alpha_t     = 0.001

model          = utils.GNLMRegression("LinearRegression",copy.deepcopy(layers_regress))
loss           = torch.nn.MSELoss(reduction='mean')
optimizer      = torch.optim.SGD(model.parameters(), lr=alpha_t, momentum=0.0)
monitor        = utils.MyMonitorTest(model,loss,dl_train,dl_test,nbmax_epoqs,debug_out,device)

loss_train_s,tmax,monistopc = \
    utils.f_train_glmr(dl_train,model,optimizer,loss,monitor,device=device,
                       loss_yy_model=loss_yy_model,printed=2)

### Post-processing and mean square error

In [ ]:
yhat_train_l1, y_train_l1 = utils.f_get_yhat(model.cpu(),dl_train)
yhat_test_l1, y_test_l1   = utils.f_get_yhat(model.cpu(),dl_test)

y_train_l1     = y_train_l1.squeeze()
y_test_l1      = y_test_l1.squeeze()
yhat_train_l1  = yhat_train_l1.squeeze()
yhat_test_l1   = yhat_test_l1.squeeze()

In [ ]:
_,_, = utils.f_metrics_regression(y_test_l1,yhat_test_l1,True)
_,_, = utils.f_metrics_regression(y_train_l1,yhat_train_l1,True)

### Interpretation of the obtained model

<!-- <table><tr>
<td> <img src="./images/pytorch_nn_450d_lasso_without_l1.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="./images/pytorch_nn_450d_lasso_with_l1.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table> -->

### Search for optimal learning rate via a grid from parameters ranges with hdf5 files

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch

x_y_all_450d_lasso=np.load(towdir('./x_y_450d_lasso.npz'))

x = x_y_all_450d_lasso['x']
y = x_y_all_450d_lasso['y']

dataset = TensorDataset( torch.from_numpy(x.astype(np.float32)), 
                         torch.from_numpy(y.astype(np.float32)) )


In [ ]:
n_all, d_all = x.shape
print( n_all, d_all )

In [ ]:
k_fold = 5

In [ ]:
import numpy.random as rd

def f_idx_traintest_kfolds(n,k_fold=5,shuffle = True):
    if not shuffle : idx_all = range(n)
    if shuffle     : idx_all = rd.permutation(range(n))
    idx_s = dict()
    for k,idx_test in enumerate(np.array_split(idx_all,k_fold)):
        idx_train = [e for e in idx_all if e not in idx_test]
        idx_s[str(k)] = dict({"train":np.asarray(idx_train),
                              "test":np.asarray(idx_test)})
    return idx_s

In [ ]:
idx_s = f_idx_traintest_kfolds(n_all,k_fold=k_fold)

In [ ]:
alpha_t     = 0.001
lambda_l1   = 0.02
nbmax_epoqs = 350
batch_size  = 10

The call to the function is thus: 

In [ ]:
import torch.nn as nn
import copy
layers_regress = []
layers_regress.append(nn.Linear(px,1,bias=True))

model_          = utils.GNLMRegression("LinearRegression",copy.deepcopy(layers_regress))
loss_           = torch.nn.MSELoss(reduction='sum')

In [ ]:
loss_train_s_s, loss_test_s_s, yhat_train_s, \
y_train_s, yhat_test_s, y_test_s = \
    utils.f_reg_l1_nn_cv(idx_s,None,dataset,model_,loss_,batch_size,alpha_t,
                   nbmax_epoqs,debug_out,device=device,
                  )

In [ ]:
loss_train_s_s, loss_test_s_s, yhat_train_s, \
y_train_s, yhat_test_s, y_test_s = \
    utils.f_reg_l1_nn_cv(idx_s,None,dataset,model_,loss_,batch_size,alpha_t,
                   nbmax_epoqs,debug_out,device=device,
                  loss_yy_model=loss_yy_model,printed=2,
                  hyperparameter_to_print=str(f"lambda_l1={lambda_l1}")
                  )

In [ ]:
alpha_t_s = np.array([0.1,0.075,0.05,0.01,0.0075,0.005,0.003,0.001,0.00075,0.0005,0.0001])[::-1]

In [ ]:
namefile_s = utils.f_save_cvdatasets_to_h5py(idx_s,dataset,
                                       towdir("x_y__450d_lasso__"),
                                       x.shape[1],
                                       transformx=None)

In [ ]:
namefile_s

In [ ]:
alpha_t_s   = np.array([0.00005,0.000075,0.001,0.00125,0.00150])
lambda_l1_s = np.array([0.001,0.005,0.01,0.015,0.02,0.025,0.03,0.04,0.05,0.1,0.2,0.3]) #[::-1]

In [ ]:
nbmax_epoqs = 100

resu_s = []
para_s = []

for lambda_l1 in iter(lambda_l1_s):
    for alpha_t in iter(alpha_t_s):

        print(f"lambda_l1={lambda_l1} alpha_t={alpha_t}", end='')

        loss_train_s_s, loss_test_s_s, yhat_train_s, \
        y_train_s, yhat_test_s, y_test_s = \
        utils.f_reg_l1_nn_cv(idx_s,None,dataset,model_,loss_,batch_size,alpha_t,
                       nbmax_epoqs,debug_out,device=device,
                       loss_yy_model=loss_yy_model,printed=0,
                       hyperparameter_to_print=str(f"lambda_l1={lambda_l1}")
                       )
        
        mse_train_s = []
        for y_train_, yhat_train_ in iter(zip(y_train_s,yhat_train_s)):
            mse_,r2_ = utils.f_metrics_regression(y_train_,yhat_train_,False)
            mse_train_s.append(mse_)
        
        mse_test_s = []
        for y_test_, yhat_test_ in iter(zip(y_test_s,yhat_test_s)):
            mse_,r2_ = utils.f_metrics_regression(y_test_,yhat_test_,False)
            mse_test_s.append(mse_)
            
        resu_s.append([mse_train_s,mse_test_s,yhat_train_s,y_train_s,yhat_test_s,y_test_s])
        para_s.append([lambda_l1,alpha_t])
        
        msetr = round(np.mean(mse_train_s),4)
        msete = round(np.mean(mse_test_s),4)
        
        print(" mse_tr_mean=", msetr, end='')
        print(" mse_te_mean=", msete, end='\n')

In [ ]:
import pandas as pd

meanmsetrain = np.asarray([np.mean(resu_[0]) for resu_ in resu_s])
stdmsetrain  = np.asarray([np.std(resu_[0]) for resu_ in resu_s])
meanmsetest  = np.asarray([np.mean(resu_[1]) for resu_ in resu_s])
stdmsetest   = np.asarray([np.std(resu_[1]) for resu_ in resu_s])

results = [ [t[0] for t in para_s],[t[1] for t in para_s],
           meanmsetrain, meanmsetest, stdmsetrain, stdmsetest, 
           stdmsetrain/meanmsetrain, stdmsetest/meanmsetest]
results_pd = pd.DataFrame(results).transpose()
results_pd.columns = ["lambda_l1","alpha_t",
                      "mse_tr_mean", "mse_te_mean", 
                      "mse_tr_std", "mse_te_std", "mse_tr_coeffvar", 
                      "mse_te_coeffvar",]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

g = sns.lineplot(data=results_pd, x='lambda_l1', y='mse_te_mean', style='alpha_t')

# g.set(xscale='log')
g.set(xticks=results_pd['lambda_l1'])
g.set(xticklabels=results_pd['lambda_l1'])
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0, title = "alpha_t")